Osnabrück University - Computer Vision (Winter Term 2020/21) - Prof. Dr.-Ing. G. Heidemann, Ulf Krumnack, Axel Schaffland, Ludwig Schallner

# Exercise Sheet 11: Object recognition: PCA and Interest Points

## Introduction

This week's sheet should be solved and handed in before the end of **Saturday, January 30, 2021**. If you need help (and Google and other resources were not enough), feel free to contact your groups' designated tutor or whomever of us you run into first. Please upload your results to your group's Stud.IP folder.

## Exercise 1: Pattern Recognition and PCA [4 points]

**a)** What are the goals of *pattern recognition*? How can they be achieved? What are the problems?

YOUR ANSWER HERE

**b)** What is *principal component analysis*? How is it related to pattern recognition?

YOUR ANSWER HERE

**c)** Explain how principal components can be computed? Then implement a function that performs the computation.

YOUR ANSWER HERE

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np


def pca(data):
    """
    Perform principal component analysis.
    
    Args:
        data (ndarray): an array of shape (n,k),
        meaning n entries with k dimensions
        
    Returns: two arrays
        pc (ndarray): array of shape (k,k) holding the principal components in its columns.
        var (ndarray): k-vector holding the corresponding variances, in descending order.
    """
    
    # Replace the following two lines by your code ...
    pc = np.random.randn(data.shape[1],data.shape[1])
    var = np.random.rand(data.shape[1])
    # YOUR CODE HERE
    return pc, var

# generate some random data
np.random.seed(23)
data = np.random.multivariate_normal([0,0], cov = [[1, .55], [.55, .5]], size=300)

# compute the principal components
pc, var = pca(data)
mean = data.mean(axis=0)

# plot the results
plt.figure(figsize=(12,8))
plt.xlim(-4,4)
plt.ylim(-2.5,2.5)
plt.scatter(*data.T)
plt.quiver(*mean[np.newaxis].repeat(2,axis=0).T, *(np.sqrt(var)*pc), color='red', scale=1, scale_units='xy')
plt.show()

# sanity check
assert np.allclose(var, [1.216, 0.137], rtol=1e-3)

## Exercise 2: Eigenfaces [6 points]

**a)** Import the images from the directory `images/trainimgs` into an numpy array using the function 
`read_images_from_directory` provided in the cell below. Display the images and the corresponding names.

In [ ]:
%matplotlib inline
import sys
import os
import glob
import numpy as np
import matplotlib.pyplot as plt

def read_images_from_directory(directory, suffix, shape):
    """
    Read all images found in DIRECTORY with given file
    name SUFFIX. All images should have the same SHAPE,
    specified as (rows,columns).
    
    Args:
        directory (string): Name of input directory.
        suffix (string): File type suffix.
        shape (tuple): Shape of images to be loaded.
    
    Returns:
        images (ndarray): A numpy array of shape m*rows*columns (from shape)
        names (list): A list of corresponding image names.
    """

    # initialize the image array and name list
    #images = np.empty((0, *shape))
    images = np.empty((0, ) + shape)
    names = []

    # now loop through all image files in the directory
    for file_name in glob.glob(directory + os.sep + '*.' + suffix):
        if os.path.isfile(file_name):

            # load each image (as double)
            img = plt.imread(file_name)

            # check for correct size
            if img.shape == shape:
                images = np.append(images, img.reshape((1, ) + shape), axis=0)
                names.append(os.path.basename(file_name))
            else:
                print(
                    'warning: Image "' + file_name +
                    '" with wrong size will be ignored!',
                    file=sys.stderr)

    return images, names


# image file suffix
suffix = 'pgm'

# image size
img_shape = (192, 168)

# YOUR CODE HERE

**b)** Use PCA to compute the eigenfaces (i.e. the eigenvectors of the face images). You may use your PCA function from Exercise 1c or some build in function. Explain what kind of input PCA expects, and how that fits to our images (you may have to `reshape` the images!). Finally, display the eigenfaces.

In [ ]:
# YOUR CODE HERE

**c)** Now project the training face images into the eigenspace to calculate their ”feature vectors”,
i.e. a representation with significantly lower dimension. For the projection of the face images,
they have to be centered first, i.e. the mean face vector has to be subtracted. Store the mean face in some vector (`mean_face`) and the representation achieved in some array (`face_db`). Finally restore the images from `face_db` and display them alongside the original image. Try out the effect of changing the number of eigenfaces to be used (`num_eigenfaces`).

In [ ]:
# number of eigenfaces to be used
num_eigenfaces = 19

# Remark: a value of 20 (theoretical perfect reconstruction) may suffer from numerical
# instability (the last eigenface may introduce noise).
# However, a value of 19 suffices for almost perfect reconstruction ...

# YOUR CODE HERE

**d)** Implement the function `recognize_face` that recognizes a face from that database by calculating the euclidean distance of this face feature vector to all of the training feature vectors from the database. The feature vector with the smallest distance represents the winner category. Check your implementation by recognizing the images from the training set (they should be recognized without error).

In [ ]:
from scipy.spatial.distance import cdist

def recognize_face(face, eigenfaces, mean_face, face_db):
    """
    Recognize a face from a face database.
    and return the index of the best matching database entry.

    The FACE is first centered and projected into the eigeface
    space provided by EIGENFACES. Then the best match is found
    according to the euclidean distance in the eigenface space.
    
    Args:
        face (ndarray): Face to be recognised.
        eigenfaces (ndarray): Array of eigenfaces.
        mean_face (ndarray): Average face.
        face_db (ndarray): Database of faces projectected into Eigenface space.
        
    Returns:
        index (uint): Position of the best matching face in face_db.
    """
    index = -1

    # YOUR CODE HERE

    return index


# ... and now check your function on the training set ...
# YOUR CODE HERE

**e)** Now classify the images in directory `images/testimg/`. Try to reduce the number of principal components
used. How many PCs are necessary to still achieve perfect classification?

In [ ]:
test_imgs, test_names = read_images_from_directory('images/testimg', suffix,
                                                   img_shape)

# YOUR CODE HERE

## Exercise 3: Local features and interest points [4 points]

**a)** Explain in your own words: What are *local features*? How are they used?

YOUR ANSWER HERE

**b)** What are *interest points* and what are they used for? What properties are desirable?

YOUR ANSWER HERE

## Exercise 4: Computing interest points [6 points]


**a)** Explain in your own words the idea of the Moravec IP operator. What are its properties? Implement this IP operator and apply it to the image `lighthouse.png`. Try different window sizes and threshold values.

YOUR ANSWER HERE

In [ ]:
%matplotlib inline
import imageio
import numpy as np
import scipy.ndimage as nd
import matplotlib.pyplot as plt

img = imageio.imread('images/lighthouse.png', pilmode='F')

window_size = 3
threshold = .2

def moravec(img, window_size):
    """Moravec corner detector.
    
    Arguments
    ---------
    img: np.ndarray
        The input image
    window_size: int
        The size of the window to consider

    Results
    -------
    response: np.ndarray
        Response (of the same size as img), indicating interest points.
    """
    response = np.zeros_like(img)
    # YOUR CODE HERE
    return response

#response = moravec(img, window_size)
response = moravec2(img, window_size)

thresholded = np.zeros_like(response)
thresholded[response > threshold] = 1

plt.figure(figsize=(12,8))
plt.subplot(2,2,1); plt.imshow(img, cmap='gray'); plt.title('Original')
plt.subplot(2,2,2); plt.imshow(response, cmap='viridis'); plt.title('Moravec "heatmap"')
plt.colorbar()
plt.subplot(2,2,3); plt.imshow(thresholded, cmap='gray'); plt.title(f'Thresholded (>{threshold})')
plt.subplot(2,2,4); plt.imshow(img[1:-1,:-1], cmap='gray'); plt.title('Corners')
mask = np.zeros(thresholded.shape + (4,), dtype=np.int)
mask[:,:,0] = 255
mask[:,:,3] = thresholded*255
plt.imshow(mask, interpolation='none')
plt.tight_layout()
plt.show()

**b)** How does the Harris corner detector work and in what sense does it improve the Moravec IP operator. Implement the Harris corner detector and apply it to `lighthouse.png`. Again, try different "window sizes" (values for $\sigma$).

YOUR ANSWER HERE

In [ ]:
%matplotlib inline
import imageio
import numpy as np
import matplotlib.pyplot as plt
import scipy.ndimage as nd

k = .04
window_size = 3

img = imageio.imread('images/lighthouse.png', pilmode='F')/255.

def harris(img, window_size=3, k=0.04):
    """Harris corner detector.
    
    Arguments
    ---------
    img: np.ndarray
        The input image
    window_size: int
        The size of the window to consider
    k: float
        The parameter k
        
    Results
    -------
    response: np.ndarray
        Response (of the same size as img), indicating interest points.
    """
    response = np.zeros_like(img)
    # YOUR CODE HERE
    return response

response = harris(img, window_size, k)
corners = response.copy()        
corners[response<0] = 0

plt.figure(figsize=(12,12))
plt.subplot(2,2,1); plt.imshow(img, cmap='gray'); plt.title("Original")
plt.subplot(2,2,2); plt.imshow(response, cmap='viridis'); plt.title('Harris "heatmap"')
plt.colorbar()
plt.subplot(2,2,3); plt.imshow((corners**.25), cmap='gray'); plt.title(f"Thresholded (<0)")
plt.subplot(2,2,4); plt.imshow(img, cmap='gray'); plt.title('Corners')
mask = np.zeros(corners.shape + (4,), dtype=np.int)
mask[:,:,0] = 255
mask[:,:,3] = (response>0)*255
plt.imshow(mask, interpolation='none')
plt.tight_layout()
plt.show()